In [37]:
import sys
import tensorflow as tf
sys.path.insert(0,'../tf-library/')
sys.path.insert(0,'../../text-classification//')
# use transfer model

from rcalgo.tf.tfmodels.image.utils import image_common
from rcalgo.tf.tfmodels.image.transfer.big_transfer_model import BigTransferModel



In [ ]:
batch_size = 128
eval_every = len(images) // batch_size
parser = bit_common.argparser(None)
args = parser.parse_args(args=[
        '--name', 'cifar10',
        '--model', 'BiT-M-R50x1',
        '--batch', str(batch_size),
        '--eval_every', str(eval_every),
        '--save_path', './ckpt/cifar10',
        '--logdir', './cifar_transfer_log',
        '--bit_pretrained_dir', '../transfer/big_transfer/pre_trained_models/',
        '--base_lr', '0.001',   # basically no need to tune the learning rate 
                                # some options batch_size 512 lr 0.003  batch_size 128 lr 0.001
        '--gpu_num', '4' # horovod is used if supported the keras multi_gpu_model here is slow...
    ]
)

In [54]:
# prepare train data & test_data

# support two types: the numpy array or file paths  (set 'is_numpy' flag)

# numpy array example
train_data = dict()
train_data['image'] = images
train_data['label'] = labels
train_data['num_classes'] = 10
train_data['resolution'] = (32, 32)  # just approximate
train_data['is_numpy'] = True


test_data = dict()
test_data['image'] = test_images
test_data['label'] = test_labels
test_data['resolution'] = (32, 32)   # just approximate
test_data['num_classes'] = 10
test_data['is_numpy'] = False

# file paths example
train_data = dict()
train_data['image'] = file_paths
train_data['label'] = labels
train_data['num_classes'] = 10
train_data['resolution'] = (32, 32)  # just approximate
train_data['is_numpy'] = True


test_data = dict()
test_data['image'] = test_file_paths
test_data['label'] = test_labels
test_data['resolution'] = (32, 32)   # just approximate
test_data['num_classes'] = 10
test_data['is_numpy'] = False


In [55]:
# Keras model or single gpu train model
model = BigTransferModel(args)
model.run(train_data=train_data, test_data=test_data)

# Horovod train
# copy all code into A.py file 
# run in cmd line:   "horovodrun -np {gpu_num} python A.py"

In [ ]:
# setup model
# model checkpoint will be saved in  ./ckpt/cifar10-epochxx
# the saved ckpt graph is compatible with our tf-libraray format (with a input in 'input_dict' and output in 'output_dict')
# read ckpt
job = zk_tools.create_job('./ckpt/', 'cifar10-epoch1')
job.export_freeze_graph('pb_test', '1')
# everything is done